In [37]:
atracoes = [
	"Torre Eiffel",
	"Museu do Louvre",
	"Catedral de Notre-Dame",
	"Sacré-Cœur",
	"Arco do Triunfo",
	"Jardim de Luxemburgo",
	"Sainte-Chapelle",
	"Place de la Concorde",
	"Panthéon",
	"Museu d'Orsay",
	"Stade de France",
	"Parc des Princes",
]

In [38]:
localizacao_atracoes = {
	"Torre Eiffel": (48.85837, 2.29448),
	"Museu do Louvre": (48.86115, 2.33929),
	"Catedral de Notre-Dame": (48.85329, 2.34449),
	"Sacré-Cœur": (48.88722, 2.3225),
	"Arco do Triunfo": (48.86889, 2.29448),
	"Jardim de Luxemburgo": (48.85661, 2.35222),
	"Sainte-Chapelle": (48.85329, 2.34449),
	"Place de la Concorde": (48.86667, 2.33333),
	"Panthéon": (48.85333, 2.34667),
	"Museu d'Orsay": (48.86667, 2.33333),
	"Stade de France": (48.83418, 2.33595),
	"Parc des Princes": (48.82761, 2.29809),
}

In [39]:
custo_entrada_atracoes = {
	"Torre Eiffel": 25,
	"Museu do Louvre": 17,
	"Catedral de Notre-Dame": 0,
	"Sacré-Cœur": 0,
	"Arco do Triunfo": 0,
	"Jardim de Luxemburgo": 0,
	"Sainte-Chapelle": 12,
	"Place de la Concorde": 0,
	"Panthéon": 8,
	"Museu d'Orsay": 17,
	"Stade de France": 15,
	"Parc des Princes": 12,
}

In [40]:
tempo_recomendado_atracoes = {
	"Torre Eiffel": 3,
	"Museu do Louvre": 5,
	"Catedral de Notre-Dame": 2,
	"Sacré-Cœur": 1,
	"Arco do Triunfo": 1,
	"Jardim de Luxemburgo": 3,
	"Sainte-Chapelle": 2,
	"Place de la Concorde": 3,
	"Panthéon": 1,
	"Museu d'Orsay": 2,
	"Stade de France": 2,
	"Parc des Princes": 2,
}

In [41]:
!pip install folium

In [42]:
import folium

mapa = folium.Map(location = [48.856614, 2.352222], zoom_start = 12)

for atracao in atracoes:
	popup = folium.Popup(f'''<b>{atracao}</b><br>
	Tempo recomendado: {tempo_recomendado_atracoes[atracao]} hora(s)<br>
	Valor da entrada: € {custo_entrada_atracoes[atracao]}
	''',
	max_width = 200,
	sticky = True)
	folium.Marker(localizacao_atracoes[atracao], popup = popup).add_to(mapa)

mapa

In [43]:
import pyomo.environ as pyo

In [61]:
modelo = pyo.ConcreteModel()

tempo_maximo = 6
num_min_atracoes = 4

modelo = pyo.ConcreteModel()

modelo.pontos_turisticos = pyo.Set(initialize = atracoes)

modelo.tempo_maximo = pyo.Param(initialize = tempo_maximo)
modelo.num_min_atracoes = pyo.Param(initialize = num_min_atracoes)
modelo.custo_entrada = pyo.Param(modelo.pontos_turisticos, initialize = custo_entrada_atracoes)
modelo.tempo_recomendado = pyo.Param(modelo.pontos_turisticos, initialize = tempo_recomendado_atracoes)

modelo.visitas = pyo.Var(atracoes, domain = pyo.NonNegativeReals, bounds = (0, 1))

modelo.obj = pyo.Objective(
    expr = sum(modelo.custo_entrada[ponto]*modelo.visitas[ponto] for ponto in atracoes),
    sense = pyo.minimize
)

modelo.restr_num_min_visitas = pyo.Constraint(
	expr = sum(modelo.visitas[ponto] for ponto in atracoes) >= modelo.num_min_atracoes
)

modelo.restr_tempo = pyo.Constraint(
	expr = sum(modelo.tempo_recomendado[ponto]*modelo.visitas[ponto] for ponto in atracoes) <= modelo.tempo_maximo
)

In [62]:
solver = pyo.SolverFactory('glpk')
resultado = solver.solve(modelo, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp72doit1g.glpk.raw --wglp /tmp/tmpifwoso5i.glpk.glp --cpxlp
 /tmp/tmprggzpb4d.pyomo.lp
Reading problem data from '/tmp/tmprggzpb4d.pyomo.lp'...
2 rows, 12 columns, 24 non-zeros
58 lines were read
Writing problem data to '/tmp/tmpifwoso5i.glpk.glp'...
62 lines were written
GLPK Simplex Optimizer 5.0
2 rows, 12 columns, 24 non-zeros
Preprocessing...
2 rows, 12 columns, 24 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  5.000e+00  ratio =  5.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
      0: obj =   0.000000000e+00 inf =   4.000e+00 (1)
      5: obj =   3.300000000e+01 inf =   0.000e+00 (0)
*    10: obj =   4.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/tmp72doit1g.glpk.raw'...
23 lines were written


In [63]:
print(f'Função objetivo: {pyo.value(modelo.obj)}')

for atracao in atracoes:
	valor = round(pyo.value(modelo.visitas[atracao]), 2)
	if valor>0:
		print(f'{atracao}: Visitado = {valor}')

Função objetivo: 4.0
Catedral de Notre-Dame: Visitado = 1.0
Sacré-Cœur: Visitado = 1.0
Arco do Triunfo: Visitado = 1.0
Jardim de Luxemburgo: Visitado = 0.5
Panthéon: Visitado = 0.5


In [65]:
def criar_modelo(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes):

	modelo = pyo.ConcreteModel()

	modelo.pontos_turisticos = pyo.Set(initialize = atracoes)

	modelo.tempo_maximo = pyo.Param(initialize = tempo_maximo)
	modelo.num_min_atracoes = pyo.Param(initialize = num_min_atracoes)
	modelo.custo_entrada = pyo.Param(modelo.pontos_turisticos, initialize = custo_entrada_atracoes)
	modelo.tempo_recomendado = pyo.Param(modelo.pontos_turisticos, initialize = tempo_recomendado_atracoes)

	modelo.visitas = pyo.Var(atracoes, domain = pyo.NonNegativeReals, bounds = (0, 1))

	modelo.obj = pyo.Objective(
		expr = sum(modelo.custo_entrada[ponto]*modelo.visitas[ponto] for ponto in atracoes),
		sense = pyo.minimize
	)

	modelo.restr_num_min_visitas = pyo.Constraint(
		expr = sum(modelo.visitas[ponto] for ponto in atracoes) >= modelo.num_min_atracoes
	)

	modelo.restr_tempo = pyo.Constraint(
      expr=sum(modelo.tempo_recomendado[ponto] * modelo.visitas[ponto] for ponto in atracoes) <= modelo.tempo_maximo
  )
	
	return modelo

In [66]:
from pyomo.opt import SolverStatus, TerminationCondition

def resolver_modelo(modelo, log = False):
	solver = pyo.SolverFactory('glpk')
	resultado = solver.solve(modelo, tee = log)
	print(resultado.solver.status)
	if resultado.solver.status == SolverStatus.ok and resultado.solver.termination_condition == TerminationCondition.optimal:
		print('Solução ótima encontrada!')
		print(f'Função objetivo: {pyo.value(modelo.obj)}')

		for atracao in atracoes:
			valor = round(pyo.value(modelo.visitas[atracao]), 2)
			if valor>0:
				print(f'{atracao}: Visitado = {valor}')
	elif resultado.solver.status == SolverStatus.warning:
		print('Atenção: Problema pode não ter solução ótima')
	elif resultado.solver.termination_condition == TerminationCondition.infeasible:
		print('O modelo é inviável. Não há solução possível.')
	else:
		print(f'Solver status: {resultado.solver.status}')

In [67]:
tempo_maximo = 6
num_min_atracoes = 4
modelo = criar_modelo(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes)
resolver_modelo(modelo)

ok
Solução ótima encontrada!
Função objetivo: 4.0
Catedral de Notre-Dame: Visitado = 1.0
Sacré-Cœur: Visitado = 1.0
Arco do Triunfo: Visitado = 1.0
Jardim de Luxemburgo: Visitado = 0.5
Panthéon: Visitado = 0.5


In [68]:
def adicionar_restricoes(modelo, condicoes):
	for ponto, dic in condicoes.items():
		valor = dic['valor']
		sinal = dic['sinal']
		if sinal == '<=':
			modelo.add_component(f'restr_{ponto}', pyo.Constraint(expr = modelo.visitas[ponto] <= valor))
		elif sinal == '>=':
			modelo.add_component(f'restr_{ponto}', pyo.Constraint(expr = modelo.visitas[ponto] >= valor))
		else:
			raise ValueError('Sinal inválido. Use "<=" ou ">="')

In [69]:
tempo_maximo = 6
num_min_atracoes = 4
condicoes = {"Jardim de Luxemburgo": {'valor': 0, 'sinal':'<='}}
modelo = criar_modelo(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes)
adicionar_restricoes(modelo, condicoes)
resolver_modelo(modelo)

ok
Solução ótima encontrada!
Função objetivo: 4.0
Catedral de Notre-Dame: Visitado = 1.0
Sacré-Cœur: Visitado = 1.0
Arco do Triunfo: Visitado = 1.0
Place de la Concorde: Visitado = 0.5
Panthéon: Visitado = 0.5


In [70]:
tempo_maximo = 6
num_min_atracoes = 4
modelo = criar_modelo(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes)
resolver_modelo(modelo)

ok
Solução ótima encontrada!
Função objetivo: 4.0
Catedral de Notre-Dame: Visitado = 1.0
Sacré-Cœur: Visitado = 1.0
Arco do Triunfo: Visitado = 1.0
Jardim de Luxemburgo: Visitado = 0.5
Panthéon: Visitado = 0.5


In [71]:
modelo = criar_modelo(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes)
condicoes = {"Jardim de Luxemburgo": {'valor': 0, 'sinal':'<='},
             "Place de la Concorde": {'valor': 1, 'sinal':'>='}}
adicionar_restricoes(modelo, condicoes)
resolver_modelo(modelo)

ok
Solução ótima encontrada!
Função objetivo: 8.0
Sacré-Cœur: Visitado = 1.0
Arco do Triunfo: Visitado = 1.0
Place de la Concorde: Visitado = 1.0
Panthéon: Visitado = 1.0


In [86]:
def criar_modelo_inteiro(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes):

	modelo = pyo.ConcreteModel()

	modelo.pontos_turisticos = pyo.Set(initialize = atracoes)

	modelo.tempo_maximo = pyo.Param(initialize = tempo_maximo)
	modelo.num_min_atracoes = pyo.Param(initialize = num_min_atracoes)
	modelo.custo_entrada = pyo.Param(modelo.pontos_turisticos, initialize = custo_entrada_atracoes)
	modelo.tempo_recomendado = pyo.Param(modelo.pontos_turisticos, initialize = tempo_recomendado_atracoes)

	modelo.visitas = pyo.Var(atracoes, domain = pyo.Binary)

	modelo.obj = pyo.Objective(
		expr = sum(modelo.custo_entrada[ponto]*modelo.visitas[ponto] for ponto in atracoes),
		sense = pyo.minimize
	)

	modelo.restr_num_min_visitas = pyo.Constraint(
		expr = sum(modelo.visitas[ponto] for ponto in atracoes) >= modelo.num_min_atracoes
	)

	modelo.restr_tempo = pyo.Constraint(
		expr = sum(modelo.tempo_recomendado[ponto]*modelo.visitas[ponto] for ponto in atracoes) <= modelo.tempo_maximo
	)

	# modelo.restr_E = pyo.Constraint(expr=  modelo.visitas["Torre Eiffel"] + modelo.visitas["Jardim de Luxemburgo"] == 2)
	# modelo.restr_OU = pyo.Constraint(expr=  modelo.visitas["Parc des Princes"] + modelo.visitas["Stade de France"] <= 1)
	# modelo.restr_SE_ENTAO = pyo.Constraint(expr=  modelo.visitas["Parc des Princes"] <=  modelo.visitas["Jardim de Luxemburgo"] )

	return modelo

In [88]:
tempo_maximo = 8
num_min_atracoes = 3
modelo = criar_modelo_inteiro(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes)
condicoes = {
	'Parc des Princes': {'valor': 1, 'sinal':'>='}
}
adicionar_restricoes(modelo, condicoes)
resolver_modelo(modelo, log=False)

ok
Solução ótima encontrada!
Função objetivo: 12.0
Catedral de Notre-Dame: Visitado = 1.0
Jardim de Luxemburgo: Visitado = 1.0
Parc des Princes: Visitado = 1.0


In [89]:
def criar_modelo_inteiro(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes, peso_restricao = 1000):

	modelo = pyo.ConcreteModel()

	modelo.pontos_turisticos = pyo.Set(initialize = atracoes)

	modelo.tempo_maximo = pyo.Param(initialize = tempo_maximo)
	modelo.custo_entrada = pyo.Param(modelo.pontos_turisticos, initialize = custo_entrada_atracoes)
	modelo.tempo_recomendado = pyo.Param(modelo.pontos_turisticos, initialize = tempo_recomendado_atracoes)

	modelo.visitas = pyo.Var(atracoes, domain = pyo.Binary)

	modelo.obj = pyo.Objective(
		expr = sum(modelo.custo_entrada[ponto]*modelo.visitas[ponto] for ponto in atracoes)+
		peso_restricao*(num_min_atracoes-sum(modelo.visitas[ponto] for ponto in atracoes)),
		sense = pyo.minimize
	)

	modelo.restr_tempo = pyo.Constraint(
		expr = sum(modelo.tempo_recomendado[ponto]*modelo.visitas[ponto] for ponto in atracoes) <= modelo.tempo_maximo
	)

	return modelo

tempo_maximo = 8
num_min_atracoes = 8
peso_restricao = 10
modelo = criar_modelo_inteiro(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo, num_min_atracoes, peso_restricao)
resolver_modelo(modelo, log = False)

ok
Solução ótima encontrada!
Função objetivo: 38.0
Catedral de Notre-Dame: Visitado = 1.0
Sacré-Cœur: Visitado = 1.0
Arco do Triunfo: Visitado = 1.0
Place de la Concorde: Visitado = 1.0
Panthéon: Visitado = 1.0


In [91]:
def criar_modelo_inteiro_dias(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo_diario, orcamento_diario, dias):

	modelo = pyo.ConcreteModel()

	modelo.pontos_turisticos = pyo.Set(initialize=atracoes)
	modelo.dias = pyo.Set(initialize = dias)

	modelo.custo_entrada = pyo.Param(modelo.pontos_turisticos, initialize=custo_entrada_atracoes)
	modelo.tempo_recomendado = pyo.Param(modelo.pontos_turisticos, initialize=tempo_recomendado_atracoes)
	modelo.tempo_maximo_diario = pyo.Param(modelo.dias, initialize = tempo_maximo_diario)
	modelo.orcamento_diario = pyo.Param(modelo.dias, initialize = orcamento_diario)

	modelo.visitas = pyo.Var(modelo.pontos_turisticos, modelo.dias, domain = pyo.Binary)

	modelo.obj = pyo.Objective(
		expr= sum(modelo.visitas[ponto, dia] for ponto in atracoes for dia in dias),
		sense = pyo.maximize
	)

	modelo.restr_tempo_diario = pyo.Constraint(
		modelo.dias,
		rule= lambda modelo, dia: sum(modelo.tempo_recomendado[ponto]*modelo.visitas[ponto, dia] for ponto in atracoes) <= modelo.tempo_maximo_diario[dia]
	)

	modelo.restr_orcamento_diario = pyo.Constraint(
		modelo.dias,
		rule= lambda modelo, dia: sum(modelo.custo_entrada[ponto]*modelo.visitas[ponto, dia] for ponto in atracoes) <= modelo.orcamento_diario[dia]
	)

	modelo.restr_visita_unica = pyo.Constraint(
		modelo.pontos_turisticos,
		rule= lambda modelo, ponto: sum(modelo.visitas[ponto, dia] for dia in modelo.dias) <= 1
	)

	return modelo

def resolver_modelo_dias(modelo, log = False):
	solver = pyo.SolverFactory('glpk')
	resultado = solver.solve(modelo, tee = log)
	print(resultado.solver.status)
	if resultado.solver.status == SolverStatus.ok and resultado.solver.termination_condition == TerminationCondition.optimal:
		print('Solução ótima encontrada!')
		print(f'Função objetivo: {round(pyo.value(modelo.obj), 2)}')
		for dia in modelo.dias:
			for atracao in modelo.pontos_turisticos:
				valor_variavel_decisao = round(pyo.value(modelo.visitas[atracao, dia]), 2)
				if valor_variavel_decisao > 0:
					print(f'No dia {dia} visitar {atracao}')
	elif resultado.solver.status == SolverStatus.warning:
		print('Atenção: Problema pode não ter solução ótima')
	elif resultado.solver.termination_condition == TerminationCondition.infeasible:
		print('O modelo é inviável. Não há solução possível.')
	else:
		print(f'Solver status: {resultado.solver.status}')

In [92]:
tempo_maximo_diario = {1:6, 2:6, 3:6, 4:6}
orcamento_diario = {1:100, 2:100, 3:100, 4:100}
dias = [1,2,3,4]
modelo = criar_modelo_inteiro_dias(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo_diario, orcamento_diario, dias)
resolver_modelo_dias(modelo, log = False)

ok
Solução ótima encontrada!
Função objetivo: 11.0
No dia 1 visitar Torre Eiffel
No dia 1 visitar Parc des Princes
No dia 2 visitar Catedral de Notre-Dame
No dia 2 visitar Sacré-Cœur
No dia 2 visitar Arco do Triunfo
No dia 2 visitar Stade de France
No dia 3 visitar Jardim de Luxemburgo
No dia 3 visitar Sainte-Chapelle
No dia 4 visitar Place de la Concorde
No dia 4 visitar Panthéon
No dia 4 visitar Museu d'Orsay


In [93]:
def criar_modelo_inteiro_dias(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo_diario, orcamento_diario, dias):

	modelo = pyo.ConcreteModel()

	modelo.pontos_turisticos = pyo.Set(initialize=atracoes)
	modelo.dias = pyo.Set(initialize = dias)

	modelo.custo_entrada = pyo.Param(modelo.pontos_turisticos, initialize=custo_entrada_atracoes)
	modelo.tempo_recomendado = pyo.Param(modelo.pontos_turisticos, initialize=tempo_recomendado_atracoes)
	modelo.tempo_maximo_diario = pyo.Param(modelo.dias, initialize = tempo_maximo_diario)
	modelo.orcamento_diario = pyo.Param(modelo.dias, initialize = orcamento_diario)

	modelo.visitas = pyo.Var(modelo.pontos_turisticos, modelo.dias, domain = pyo.Binary)
	modelo.menor_qtd_atracoes = pyo.Var(within= pyo.NonNegativeIntegers)

	modelo.obj = pyo.Objective(
		expr= modelo.menor_qtd_atracoes,
		sense = pyo.maximize
	)

	modelo.restr_tempo_diario = pyo.Constraint(
		modelo.dias,
		rule= lambda modelo, dia: sum(modelo.tempo_recomendado[ponto]*modelo.visitas[ponto, dia] for ponto in atracoes) <= modelo.tempo_maximo_diario[dia]
	)

	modelo.restr_orcamento_diario = pyo.Constraint(
		modelo.dias,
		rule= lambda modelo, dia: sum(modelo.custo_entrada[ponto]*modelo.visitas[ponto, dia] for ponto in atracoes) <= modelo.orcamento_diario[dia]
	)

	modelo.restr_visita_unica = pyo.Constraint(
		modelo.pontos_turisticos,
		rule= lambda modelo, ponto: sum(modelo.visitas[ponto, dia] for dia in modelo.dias) <= 1
	)

	modelo.restr_menor_qtd_atracoes = pyo.Constraint(
		modelo.dias,
		rule = lambda modelo, dia: sum(modelo.visitas[ponto,dia] for ponto in atracoes) >= modelo.menor_qtd_atracoes
	)

	return modelo

In [95]:
tempo_maximo_diario = {1:6, 2:6, 3:6, 4:6}
orcamento_diario = {1:100, 2:100, 3:100, 4:100}
dias = [1,2,3,4]
modelo = criar_modelo_inteiro_dias(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo_diario, orcamento_diario, dias)
resolver_modelo_dias(modelo, log = False)

ok
Solução ótima encontrada!
Função objetivo: 2.0
No dia 1 visitar Jardim de Luxemburgo
No dia 1 visitar Panthéon
No dia 2 visitar Catedral de Notre-Dame
No dia 2 visitar Museu d'Orsay
No dia 2 visitar Stade de France
No dia 3 visitar Sacré-Cœur
No dia 3 visitar Sainte-Chapelle
No dia 3 visitar Place de la Concorde
No dia 4 visitar Torre Eiffel
No dia 4 visitar Arco do Triunfo
No dia 4 visitar Parc des Princes


In [96]:
import random

def gerar_parametros(num_atracoes, num_dias):

	atracoes = range(1, num_atracoes)
	dias = range(1, num_dias+ 1)

	custo_entrada_atracoes = {ponto_turistico: random.randint(10, 100) for ponto_turistico in atracoes}
	tempo_recomendado_atracoes = {ponto_turistico: random.randint(1, 6) for ponto_turistico in atracoes}

	tempo_maximo_diario = {dia: random.randint(8, 10) for dia in dias}
	orcamento_diario = {dia: random.randint(450, 500) for dia in dias}

	return atracoes, dias, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo_diario, orcamento_diario

In [97]:
num_atracoes = 10000
num_dias = 60
atracoes, dias, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo_diario, orcamento_diario = gerar_parametros(num_atracoes = num_atracoes, num_dias = num_dias)

modelo = criar_modelo_inteiro_dias(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo_diario, orcamento_diario, dias)
resolver_modelo_dias(modelo)

ok
Solução ótima encontrada!
Função objetivo: 8.0
No dia 1 visitar 714
No dia 1 visitar 875
No dia 1 visitar 903
No dia 1 visitar 1348
No dia 1 visitar 2815
No dia 1 visitar 3163
No dia 1 visitar 3910
No dia 1 visitar 5893
No dia 2 visitar 187
No dia 2 visitar 1134
No dia 2 visitar 1630
No dia 2 visitar 2215
No dia 2 visitar 3091
No dia 2 visitar 3450
No dia 2 visitar 5476
No dia 2 visitar 6052
No dia 3 visitar 18
No dia 3 visitar 122
No dia 3 visitar 881
No dia 3 visitar 1771
No dia 3 visitar 4123
No dia 3 visitar 4467
No dia 3 visitar 4482
No dia 3 visitar 5844
No dia 4 visitar 313
No dia 4 visitar 1564
No dia 4 visitar 1669
No dia 4 visitar 1961
No dia 4 visitar 3278
No dia 4 visitar 3824
No dia 4 visitar 5514
No dia 4 visitar 5594
No dia 5 visitar 87
No dia 5 visitar 199
No dia 5 visitar 904
No dia 5 visitar 2268
No dia 5 visitar 2721
No dia 5 visitar 3750
No dia 5 visitar 4890
No dia 5 visitar 5226
No dia 6 visitar 36
No dia 6 visitar 709
No dia 6 visitar 1791
No dia 6 visitar 312

In [98]:
def resolver_modelo_dias(modelo, gap=None, tempo_limite = None, log = False):
	solver = pyo.SolverFactory('glpk')

	if gap is not None:
		solver.options['mipgap'] = gap
	if tempo_limite is not None:
		solver.options['tmlim'] = tempo_limite

	resultado = solver.solve(modelo, tee = log)
	print(resultado.solver.status)
	if resultado.solver.status == SolverStatus.ok and resultado.solver.termination_condition == TerminationCondition.optimal:
		print('Solução ótima encontrada!')
		print(f'Função objetivo: {round(pyo.value(modelo.obj), 2)}')
		for dia in modelo.dias:
			for atracao in modelo.pontos_turisticos:
				valor_variavel_decisao = round(pyo.value(modelo.visitas[atracao, dia]), 2)
				if valor_variavel_decisao > 0:
					print(f'No dia {dia} visitar {atracao}')
	elif resultado.solver.status == SolverStatus.warning:
		print('Atenção: Problema pode não ter solução ótima')
	elif resultado.solver.termination_condition == TerminationCondition.infeasible:
		print('O modelo é inviável. Não há solução possível.')
	else:
		print(f'Solver status: {resultado.solver.status}')

In [99]:
tempo_maximo_diario = {1:6, 2:6, 3:6, 4:6}
orcamento_diario = {1:100, 2:100, 3:100, 4:100}
dias = [1,2,3,4]
modelo = criar_modelo_inteiro_dias(atracoes, custo_entrada_atracoes, tempo_recomendado_atracoes, tempo_maximo_diario, orcamento_diario, dias)
resolver_modelo_dias(modelo, log = False)

ok
Solução ótima encontrada!
Função objetivo: 6.0
No dia 1 visitar 37
No dia 1 visitar 187
No dia 1 visitar 398
No dia 1 visitar 561
No dia 1 visitar 875
No dia 1 visitar 896
No dia 2 visitar 12
No dia 2 visitar 881
No dia 2 visitar 1291
No dia 2 visitar 1375
No dia 2 visitar 2040
No dia 2 visitar 4611
No dia 3 visitar 36
No dia 3 visitar 41
No dia 3 visitar 295
No dia 3 visitar 820
No dia 3 visitar 1910
No dia 3 visitar 1930
No dia 4 visitar 20
No dia 4 visitar 87
No dia 4 visitar 1005
No dia 4 visitar 3171
No dia 4 visitar 3824
No dia 4 visitar 5349
